This jupyter notebook contains the supplementary Python Code used in the manuscript **G4-quadruplex-mediated genomic instability drives SNVs in cancer** by Richl et al.

# Contents

- 1. Libraries
- 2. Gathering datasets
- 3. Analyzing and plotting the spatial distribution of cSNVs or SNPs at G4s.
- 4. Analyzing the density distribution of G4s and cSNVs in the CDK4 gene.
- 5. Analyzing and plotting the average number of cSNVs or random SNPs in genomic 5 kbps windows containing either 0/!0/>=10 G4s.
- 6. Stratifying genomic 5 kbps windows based on the number of G4s contained and plotting the distribution of cSNVs or SNPs.
- 7. Analyzing and plotting the average number of G4s in non-cancer or cancer genes.
- 8. Analyzing the density distribution of cSNVs or SNPs in either cancer- or non-cancer genes for genetic windows with 0 / >=10 G4s.
- 9. Perform incomplete categorical bootstrapping and approximate the limit R2 for increasing random sampling sizes.
- 10. Plot incomplete categorical bootstrapping for the full dataset.
- 11. Comparing the average number of normalized cSNVs across different cancer types in genomic windows containing 0/>=10 G4s.
- 12. Group patients by cancer type and tissue and calculate median G4-averages across patients with similiar diagnoses. Plot as sunburst plot.
- 13. Group patients by cancer type and calculate intra-group mean-normalized standard deviation.
- 14. Fit Cox's proportional hazards model to patients that are eeither alive or died due to cancer-related circumstances. 
- 15. Plot results of Cox's proportional hazards model.


# 1. Libraries
The following libraries are used throughout the notebook and need to be installed.

In [ ]:
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import linear_model
import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import matplotlib.ticker as ticker
from lifelines import CoxPHFitter
from matplotlib.pyplot import figure


# Setting global options 
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
sns.set_theme()

# Set colors
cSNPs = "#e69422"
SNPs = "#1e60c9"
Random = "#000000"

# 2. Gathering Datasets
Loading the data and building one final dataframe holding all cSNVs with associated meta- and window-data.

In [ ]:
### Loading the Dataframe with all Mutations

df = pd.read_csv("~/SNP_exons", sep="\t", header=None).drop(columns={5,6,7,8,9,10}).rename(columns={0: "SNP_ID", 
                                                                                              1: "Strand", 
                                                                                              2: "Type", 
                                                                                              3: "Reference_Allele", 
                                                                                              4: "Allele", 
                                                                                              11: "case_id",
                                                                                              12: "window_id"}).drop_duplicates(keep="first")
df2 = pd.read_csv("~/exons_G4.count", sep="\t", header=None).drop(columns={0,1,2}).rename(columns={3: "window_id", 4: "G4_Count"})
df3 = pd.read_csv("~/exons_SNP.count", sep="\t", header=None).drop(columns={0,1,2}).rename(columns={3: "window_id", 4: "SNP_Count"})

df_m1 = pd.merge(df2,df, on="window_id", how="left")#.fillna(0)
df_m1["G4_Count"] = df_m1["G4_Count"].fillna(0)
df_m1["SNP_ID"] = df_m1["SNP_ID"].fillna(0)
df_m2 = pd.merge(df_m1,df3, on="window_id", how="left")#.fillna(0)
df_c = pd.read_csv("~/clinical.tsv", sep="\t")
df_c2 = df_c[["case_id", "cause_of_death", "days_to_birth", "primary_diagnosis", "therapeutic_agents", "vital_status", "tissue_or_organ_of_origin"]].drop_duplicates(subset="case_id", keep="first")
df_m3 = pd.merge(df_m2, df_c2, on="case_id", how="left")

drop_list = ["case_id", "ajcc_clinical_t", "ajcc_clinical_n", "ajcc_clinical_m", "ajcc_pathologic_stage", "ajcc_pathologic_t", "ajcc_pathologic_n", "ajcc_pathologic_m", "prior_malignancy"]
df_test = pd.merge(df_m3, df_c[drop_list], on = "case_id", how="left")
df_m3 = df_test

### Generating the Dataframe with cSNPs grouped by windows

final = df_m3.groupby(["window_id"]).mean()




## Analyzing and plotting the spatial distribution of cSNVs or SNPs at G4s.

In [ ]:
# Data Loading
df1 = pd.read_csv("~/Common_SNPs_exons_5kb_windows_G4s.Distance", delim_whitespace=True, header=None)[0].value_counts().reset_index().rename(columns={0: "Count", "index": "Distance"})
df2 = pd.read_csv("~/random_exons_5kb_windows_G4s.Distance", delim_whitespace=True, header=None)[0].value_counts().reset_index().rename(columns={0: "Count", "index": "Distance"})
df3 = pd.read_csv("~/SNPs_exons_5kb_windows_G4s.Distance", delim_whitespace=True, header=None)[0].value_counts().reset_index().rename(columns={0: "Count", "index": "Distance"})
df1["Origin"] = "SNP"
df2["Origin"] = "Random"
df3["Origin"] = "cSNV"
df4 = pd.concat([df2,df1,df3])

# Colors and group labels
colors = [cSNPs, Random, SNPs]
group_labels = ["cSNV", "Random", "SNP"]

# Plotting with Matplotlib
fig, ax = plt.subplots(figsize=(8, 8))
for label, color in zip(group_labels, colors):
    data = df4[df4["Origin"] == label]["Distance"]
    density = gaussian_kde(data)
    xs = np.linspace(-200000, 200000, 1000)
    density.covariance_factor = lambda: 0.25
    density._compute_covariance()
    ax.plot(xs, density(xs), label=label, color=color)

ax.set_xlim(-200000, 200000)
ax.set_xlabel("Distance of SNV to closest G4 in bp")
ax.set_ylabel("Normalized Density")
ax.legend(fontsize=14)

# Format x-axis labels and add minor ticks
ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, p: format(int(x/1000), ',')))
ax.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax.tick_params(which='both', width=1)
ax.tick_params(which='major', length=6)
ax.tick_params(which='minor', length=3)

plt.show()


## Analyzing the density distribution of G4s and cSNVs in the *CDK4* gene.

In [ ]:
# Data Loading

df_G4 = pd.read_csv("CDK4.G4s", sep="\t", header=None).rename(columns={0: "Chr", 1: "Start", 2: "Stop", 3: "ID"})
df_GDC = pd.read_csv("CDK4.ALL_GDC", sep="\t", header=None)[[0,1,2,3]].rename(columns={0: "Chr", 1: "Start", 2: "Stop", 3: "ID"})
df_G4["Class"] = "G4"
df_GDC["Class"] = "GDC"
df_all = pd.concat([df_G4, df_GDC])
df_all["Summit"] = (df_all["Stop"] - df_all["Start"]) / 2 + df_all["Start"]

# Plotting

sns.set(rc={'figure.figsize':(10, 4)})

sns.set(font_scale=2)
sns.set_style("white")
ax = sns.kdeplot(data=df_all[df_all["Class"] == "G4"]["Summit"], color = "#000008", bw_adjust=0.15)
sns.kdeplot(data=df_all[df_all["Class"] == "GDC"]["Summit"], bw_adjust=0.4, color = cSNPs)
sns.despine()
plt.savefig("output_seaborn_plot.png", dpi=300)


## Analyzing and plotting the average number of cSNVs or random SNPs in genomic 5 kbps windows containing either 0/!0/>=10 G4s.

In [ ]:
lower_limit = 1
test1 = final[final["G4_Count"] == 0]["SNP_Count"].sample(5000).clip(lower=lower_limit)
test2 = final[final["G4_Count"] != 0]["SNP_Count"].sample(5000).clip(lower=lower_limit)
test3 = final[final["G4_Count"] > 9]["SNP_Count"].clip(lower=lower_limit)
test4 = final_random[final_random["G4_Count"] == 0]["SNP_Count"].sample(5000).clip(lower=lower_limit)
test5 = final_random[final_random["G4_Count"] != 0]["SNP_Count"].sample(5000).clip(lower=lower_limit)
test6 = final_random[final_random["G4_Count"] > 9]["SNP_Count"].clip(lower=lower_limit)
fig = go.Figure()

# Define a helper function to add box plots with adjusted x values
def add_box_trace(fig, y, name, fillcolor, x_offset):
    fig.add_trace(go.Box(
        y=y,
        name=name,
        boxmean=True,
        boxpoints=False,
        line_color='black',
        fillcolor=fillcolor,
        opacity=0.6,
        x=[x_offset] * len(y)
    ))


add_box_trace(fig, test1, 'No G4-Hit', 'lightsteelblue', 1)
add_box_trace(fig, test2, 'G4-Hit', 'lightcoral', 2)
add_box_trace(fig, test3, '>= 10 G4s', 'lightseagreen', 3)


add_box_trace(fig, test4, 'No G4-Hit', 'lightsteelblue', 4)
add_box_trace(fig, test5, 'G4-Hit', 'lightcoral', 5)
add_box_trace(fig, test6, '>= 10 G4s', 'lightseagreen', 6)

# Update the layout
fig.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    yaxis_title="SNPs / 5 kpbs",
    xaxis_title="Groups",
    xaxis=dict(
        tickvals=[1.5, 4.5],
        ticktext=["Samples", "Control"],
        tickfont=dict(size=25)
    ),
    yaxis=dict(tickfont=dict(size=25)),
    template="simple_white",
    font=dict(size=25),
    showlegend=False
)

# Set y-axis to logarithmic scale
fig.update_yaxes(type="log")

# Show the plot
fig.show()


## Stratifying genomic 5 kbps windows based on the number of G4s contained and plotting the distribution of cSNVs or SNPs. 

In [ ]:
data = []
fontsize=15
for G4 in range(0,16,1):
    data.append(list(df[df["G4_Count"] == G4]["cSNP_Count_normalized"]))
fig, ax = plt.subplots(figsize=(10,10))
parts = plt.violinplot(data,quantiles = None,
                        showmeans=False, 
                        showmedians=False,
                        showextrema=False,
                        widths = 0.8)
for pc in parts['bodies']:
    pc.set_facecolor(cSNPs)
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)

x1 = df[df["G4_Count"] <= 18].groupby(["G4_Count"]).mean().reset_index()["G4_Count"].values
y1 = df[df["G4_Count"] <= 18].groupby(["G4_Count"]).mean().reset_index()["cSNP_Count_normalized"].values

model1 = np.poly1d(np.polyfit(x1, y1, 1))

line1 = np.linspace(min(x1), max(x1), 1000)
ax.plot(line1, model1(line1), color="black", lw = 5)

ax.scatter(df[(df["G4_Count"] < 18) & (df["G4_Count"] > 0)].groupby(["G4_Count"]).mean().reset_index()["G4_Count"], df[(df["G4_Count"] < 18) & (df["G4_Count"] > 0)].groupby(["G4_Count"]).mean().reset_index()["cSNP_Count_normalized"], color="black", marker="+", s=100,zorder=2)
ax.set_title('cSNPs')
for pos in ['right', 'top']:
    plt.gca().spines[pos].set_visible(False)
plt.rcParams.update({
    'font.size': fontsize,         # default font size
    'axes.labelsize': fontsize,    # x and y labels
    'xtick.labelsize': fontsize,   # x-axis tick labels
    'ytick.labelsize': fontsize,   # y-axis tick labels
    'legend.fontsize': fontsize,   # legend font size
    'figure.titlesize': fontsize,  # figure title font size
})
plt.gca().set_yscale('log')
ax.set_ylabel('Normalized SNPs / 5kb')
ax.set_xlabel('G4s / 5kb')
plt.ylim([0.1, 100])
plt.savefig("output_stratify_1_plot.png", dpi=500)

In [ ]:
data = []
fontsize=5
for G4 in range(0,16,1):
    data.append(list(df[df["G4_Count"] == G4]["SNP_Count_normalized"]))
fig, ax = plt.subplots(figsize=(10,10))
parts = plt.violinplot(data,quantiles = None,
                        showmeans=False, 
                        showmedians=False,
                        showextrema=False,
                        widths = 0.8)
for pc in parts['bodies']:
    pc.set_facecolor(SNPs)
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)

x1 = df[df["G4_Count"] <= 18].groupby(["G4_Count"]).mean().reset_index()["G4_Count"].values
y1 = df[df["G4_Count"] <= 18].groupby(["G4_Count"]).mean().reset_index()["SNP_Count_normalized"].values

model1 = np.poly1d(np.polyfit(x1, y1, 1))

line1 = np.linspace(min(x1), max(x1), 1000)
ax.plot(line1, model1(line1), color="black", lw = 5)

ax.scatter(df[(df["G4_Count"] < 18) & (df["G4_Count"] > 0)].groupby(["G4_Count"]).mean().reset_index()["G4_Count"], df[(df["G4_Count"] < 18) & (df["G4_Count"] > 0)].groupby(["G4_Count"]).mean().reset_index()["SNP_Count_normalized"], color="black", marker="+", s=100,zorder=2)
ax.set_title('SNPs')
for pos in ['right', 'top']:
    plt.gca().spines[pos].set_visible(False)
plt.rcParams.update({
    'font.size': fontsize,         # default font size
    'axes.labelsize': fontsize,    # x and y labels
    'xtick.labelsize': fontsize,   # x-axis tick labels
    'ytick.labelsize': fontsize,   # y-axis tick labels
    'legend.fontsize': fontsize,   # legend font size
    'figure.titlesize': fontsize,  # figure title font size
})
plt.gca().set_yscale('log')
ax.set_xlabel('G4s / 5kb')
plt.ylim([0.1, 100])
plt.savefig("output_stratify_2_plot.png", dpi=500)

## Analyzing and plotting the average number of G4s in non-cancer or cancer genes.

In [ ]:
# Import Data

df = pd.read_csv("genes_G4.count", delim_whitespace=True, header=None).rename(columns={1: "Gene", 0: "G4_Count"})
df_cg = pd.read_csv("Cancer_Genes", sep="\t").rename(columns={"Gene Symbol": "Gene"})
df_cg["Cancer_Gene"] = 1
df_m = pd.merge(df,df_cg, on="Gene", how="left").fillna(0).drop_duplicates(subset="Gene", keep="first")

# Plotting

test1 = df_m[df_m["Cancer_Gene"] == 0]["G4_Count"]
test2 = df_m[df_m["Cancer_Gene"] != 0]["G4_Count"]
fig = go.Figure()
fig.add_trace(go.Box(
    y=test1,
    name='Non-cancer gene',
    #marker_color='#FF4136',
    boxmean=True,
    boxpoints=False,
    line_color='black',
    fillcolor="lightsteelblue",
    opacity=0.6,
))
fig.add_trace(go.Box(
    y=test2,
    name='Cancer gene',
    #marker_color='#FF851B',
    boxmean=True,
    boxpoints=False,
    line_color='black',
    fillcolor="lightcoral",
    opacity=0.6,
))

fig.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)

#fig.update_yaxes(title_text="y-axis in logarithmic scale", type="log")

fig.for_each_trace(lambda t: t.update({"marker":{"size":3}}))
fig.update_yaxes(type="log")
fig.update_yaxes(title="log G4s / gene")
fig.update_layout(
    template="simple_white",
    #legend_title="Legend Title",
    yaxis = dict(tickfont = dict(size=25)),
    xaxis = dict(tickfont = dict(size=25)),
    #legend = dict(font = dict(size = 20, color = "black")),
    font=dict(size=30))
fig.show()

## Analyzing the density distribution of cSNVs or SNPs in either cancer- or non-cancer genes for genetic windows with 0 / >=10 G4s.

In [ ]:
# Data Loading

df_GDC_cancer = pd.read_csv("ALL_GDC.sorted.bed.cancer.count", header=None, delim_whitespace=True).rename(columns={1:"window_id", 0: "GDC_Count"})
df_GDC_non_cancer = pd.read_csv("ALL_GDC.sorted.bed.non_cancer.count", header=None, delim_whitespace=True).rename(columns={1:"window_id", 0: "GDC_Count"})
df_G4 = pd.read_csv("sliding_5kb_windows_hg38_exons.G4_count", header=None, sep="\t").drop(columns={0,1,2}).rename(columns={3: "window_id", 4: "G4_Count"})
df_m_GDC_cancer = pd.merge(df_GDC_cancer, df_G4, on="window_id", how="left")
df_m_GDC_non_cancer = pd.merge(df_GDC_non_cancer, df_G4, on="window_id", how="left")


df_m_GDC_cancer["log_GDC_Count"] = np.log(df_m_GDC_cancer["GDC_Count"])
df_m_GDC_non_cancer["log_GDC_Count"] = np.log(df_m_GDC_non_cancer["GDC_Count"])
test1 = df_m_GDC_cancer[df_m_GDC_cancer["G4_Count"] == 0]["log_GDC_Count"]#.sample(10000)
test2 = df_m_GDC_cancer[df_m_GDC_cancer["G4_Count"] > 10]["log_GDC_Count"]
test3 = df_m_GDC_non_cancer[df_m_GDC_non_cancer["G4_Count"] == 0]["log_GDC_Count"]
test4 = df_m_GDC_non_cancer[df_m_GDC_non_cancer["G4_Count"] > 10]["log_GDC_Count"]

fig = go.Figure()
fig.add_trace(go.Violin(
    x=test1,
    name='',
    marker_color=cSNPs,
    #fillcolor="white",
    spanmode="hard",
    side='negative',
    #opacity=1,
    points=False,
    
))
fig.add_trace(go.Violin(
    x=test2,
    name='',
    marker_color='#9e6516',
    #fillcolor="white",
    spanmode="hard",
    side='negative',
    #opacity=1,
    points=False
))

fig.add_trace(go.Violin(
    x=test3,
    name='',
    marker_color=cSNPs,
    #fillcolor="white",
    spanmode="hard",
    side='positive',
    #opacity=0.1,
    points=False
))
fig.add_trace(go.Violin(
    x=test4,
    name='',
    marker_color='#9e6516',
    #fillcolor="white",
    spanmode="hard",
    side='positive',
    #opacity=1,
    points=False
))

fig.update_layout(
    #title="Plot Title",
    yaxis_title="Cancer Genes          Normal Genes",
    #xaxis_title="Log SNPs / 5kb",
)

fig.update_traces(box_visible=False, meanline_visible=True)
#fig.update_layout(violinmode='group')
fig.update_layout(
    autosize=False,
    width=600,
    height=600,
)
fig.update_layout(
    template="simple_white",
    #legend_title="Legend Title",
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=20)),
    #legend = dict(font = dict(size = 20, color = "black")),
    font=dict(size=25))
fig.update_layout(showlegend=False)

In [ ]:
df_SNP_cancer = pd.read_csv("Common_SNPs.bed.cancer.count", header=None, delim_whitespace=True).rename(columns={1:"window_id", 0: "SNP_Count"})
df_SNP_non_cancer = pd.read_csv("Common_SNPs.bed.non_cancer.count", header=None, delim_whitespace=True).rename(columns={1:"window_id", 0: "SNP_Count"})
df_G4 = pd.read_csv("sliding_5kb_windows_hg38_exons.G4_count", header=None, sep="\t").drop(columns={0,1,2}).rename(columns={3: "window_id", 4: "G4_Count"})
df_m_SNP_cancer = pd.merge(df_SNP_cancer, df_G4, on="window_id", how="left")
df_m_SNP_non_cancer = pd.merge(df_SNP_non_cancer, df_G4, on="window_id", how="left")


df_m_SNP_cancer["log_SNP_Count"] = np.log(df_m_SNP_cancer["SNP_Count"])
df_m_SNP_non_cancer["log_SNP_Count"] = np.log(df_m_SNP_non_cancer["SNP_Count"])
test1 = df_m_SNP_cancer[df_m_SNP_cancer["G4_Count"] == 0]["log_SNP_Count"]#.sample(10000)
test2 = df_m_SNP_cancer[df_m_SNP_cancer["G4_Count"] > 10]["log_SNP_Count"]
test3 = df_m_SNP_non_cancer[df_m_SNP_non_cancer["G4_Count"] == 0]["log_SNP_Count"]
test4 = df_m_SNP_non_cancer[df_m_SNP_non_cancer["G4_Count"] > 10]["log_SNP_Count"]

fig = go.Figure()
fig.add_trace(go.Violin(
    x=test1,
    name='',
    marker_color=SNPs,
    spanmode="hard",
    side='negative',
    points=False
))
fig.add_trace(go.Violin(
    x=test2,
    name='',
    marker_color='#123c80',
    spanmode="hard",
    side='negative',
    points=False
))
fig.add_trace(go.Violin(
    x=test3,
    name='',
    marker_color=SNPs,
    spanmode="hard",
    side='positive',
    points=False
))
fig.add_trace(go.Violin(
    x=test4,
    name='',
    marker_color='#123c80',
    spanmode="hard",
    side='positive',
    points=False
))

fig.update_layout(
    #title="SNPs",
    yaxis_title="Cancer Genes          Normal Genes",
    #xaxis_title="Log SNPs / 5kb",
)

fig.update_traces(box_visible=False, meanline_visible=True)
#fig.update_layout(violinmode='group')
fig.update_layout(
    autosize=False,
    width=600,
    height=600,
)
fig.update_layout(
    template="simple_white",
    #legend_title="Legend Title",
    yaxis = dict(tickfont = dict(size=25)),
    xaxis = dict(tickfont = dict(size=20)),
    #legend = dict(font = dict(size = 20, color = "black")),
    font=dict(size=30))
fig.update_layout(showlegend=False)

## Perform incomplete categorical bootstrapping and approximate the limit R2 for increasing random sampling sizes.

In [ ]:

key1 = "SNP_Count"
key2 = "cSNP_Count"
df_results = pd.DataFrame({"Sample_Size": [],
                           "R2_1": [],
                           "R2_2": [],
                           "Coef_1": [],
                           "Coef_2": []})

# Incomplete Categorical Bootsrapping
start = 0
stop = 16
inc = 1
tolerance = 0.3
permutation = 200
for sample in range(1,101,5):
    df_mean2 = df.sample(1)
    for G4 in range(start,stop,inc):
        i = 0
        try: 
            while i < permutation: 
                df_now = df[(df["G4_Count"] == G4)].sample(sample).mean().reset_index().T
                new_header = df_now.iloc[0] #grab the first row for the header
                df_now = df_now[1:] #take the data less the header row
                df_now.columns = new_header #set the header row as the df header
                df_mean2 = pd.concat([df_mean2, df_now])

                i += 1
        except:
            continue
    
    y1 = df_mean2[key1].fillna(0).values
    y2 = df_mean2[key2].fillna(0).values
    X = df_mean2['G4_Count'].fillna(0).values.reshape(-1,1)

    ols1 = linear_model.LinearRegression()
    model1 = ols1.fit(X, y1)
    r2_1 = model1.score(X, y1)
    
    ols2 = linear_model.LinearRegression()
    model2 = ols2.fit(X, y2)
    r2_2 = model2.score(X, y2)
    
    
    df_now = pd.DataFrame({"Sample_Size": [sample],
                           "R2_1": [r2_1],
                           "R2_2": [r2_2],
                           "Coef_1": [model1.coef_],
                           "Coef_2": [model2.coef_]})
    df_results = pd.concat([df_results, df_now])

# create figure and axis objects with subplots()

fig,ax = plt.subplots()

fig.set_size_inches(6, 6)
# make a plot
ax.plot(df_results["Sample_Size"], df_results["R2_2"], color=cSNPs, marker="o")

ax.plot(df_results["Sample_Size"], df_results["R2_1"],color=SNPs,marker="o")
ax.set_ylim([-.05, 1])


for pos in ['right', 'top']:
    plt.gca().spines[pos].set_visible(False)
ax.set_xlabel('Sample Size')
plt.rcParams.update({
    'font.size': 15,         # default font size
    'axes.labelsize': 15,    # x and y labels
    'xtick.labelsize':15,   # x-axis tick labels
    'ytick.labelsize': 15,   # y-axis tick labels
    'legend.fontsize': 25,   # legend font size
    'figure.titlesize': 40,  # figure title font size
})
ax.set_ylabel("R2 of linear fit",fontsize=25)
ax.set_xlabel("Sample size",fontsize=25)

plt.tight_layout()
plt.show()


## Plot incomplete categorical bootstrapping for the full dataset.

In [ ]:
key1 = "SNP_Count_normalized"#_Variance_normalized"
key2 = "cSNP_Count_normalized"#_Variance_normalized"
df_results = pd.DataFrame({"Sample_Size": [],
                           "R2_1": [],
                           "R2_2": [],
                           "Coef_1": [],
                           "Coef_2": []})
start = 0
stop = 16
inc = 1
permutation = 250
c = cSNPs
figure(figsize=(5,5), dpi=500)
for sample in range(1,102,5):
    data = []
    df_mean2 = df.sample(1)


    for G4 in range(start,stop,inc):
        i = 0
        try: 
            while i < permutation: 
                df_now = df[(df["G4_Count"] == G4)].sample(sample).mean().reset_index().T
                new_header = df_now.iloc[0] #grab the first row for the header
                df_now = df_now[1:] #take the data less the header row
                df_now.columns = new_header #set the header row as the df header
                df_mean2 = pd.concat([df_mean2, df_now])

                i += 1
        except:
            continue
    
    #y1 = df_mean2[key1].fillna(0).values
    y2 = df_mean2[key2].fillna(0).values
    X = df_mean2['G4_Count'].fillna(0).values.reshape(-1,1)

    #ols1 = linear_model.LinearRegression()
    #model1 = ols1.fit(X, y1)
    #r2_1 = model1.score(X, y1)
    
    ols2 = linear_model.LinearRegression()
    model2 = ols2.fit(X, y2)
    r2_2 = model2.score(X, y2)
    
    
    df_now = pd.DataFrame({"Sample_Size": [sample],
                           #"R2_1": [r2_1],
                           "R2_2": [r2_2],
                           #"Coef_1": [model1.coef_],
                           "Coef_2": [model2.coef_]})
    df_results = pd.concat([df_results, df_now])

    for G4 in range(start, stop, inc): 
        data.append(df_mean2[df_mean2["G4_Count"] == G4][key2])
    boxplot_dict = plt.boxplot(data, showfliers=False, whis=0, showcaps=False, patch_artist=True,
            medianprops=dict(color="black"),)
    for b in boxplot_dict['boxes']:
        b.set_alpha(0.1)
        b.set(color='black', linewidth=1)
        b.set(facecolor = cSNPs)
for pos in ['right', 'top']:
    plt.gca().spines[pos].set_visible(False)
plt.rcParams.update({
    'font.size': 15,         # default font size
    'axes.labelsize': 15,    # x and y labels
    'xtick.labelsize':15,   # x-axis tick labels
    'ytick.labelsize': 15,   # y-axis tick labels
    'legend.fontsize': 25,   # legend font size
    'figure.titlesize': 40,  # figure title font size
})
plt.ylabel("Normalized cSNPs / 5kb",fontsize=15)
plt.xlabel('G4s / 5kb',fontsize=15)
plt.savefig("output_bootstrap_2_plot.png", dpi=500)


## Comparing the average number of normalized cSNVs across different cancer types in genomic windows containing 0/>=10 G4s. 

In [ ]:
test1 = df_cancer[df_cancer["Patient_Count"] > 14]["0_n"]
test2 = df_cancer[df_cancer["Patient_Count"] > 14]["10_n"]
fig = go.Figure()
fig.add_trace(go.Box(
    y=test1,
    name='No G4s',
    boxmean=True,
    line_color='black',
    fillcolor="lightsteelblue",
    opacity=0.6,
    boxpoints="all"
))
fig.add_trace(go.Box(
    y=test2,
    name='>= 10 G4s',
    boxmean=True,
    boxpoints="all",
    line_color='black',
    fillcolor="lightcoral",
    opacity=0.6,
))

fig.update_layout(
    autosize=False,
    width=600,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
)
fig.update_layout(
    #title="Plot Title",
    yaxis_title="Normalized SNPs / 5 kb, grouped for cancer types",
    #legend_title="Legend Title",
    )
fig.for_each_trace(lambda t: t.update({"marker":{"size":3}}))
#fig.update_yaxes(type="log")
fig.update_yaxes(title="Cancer Types, normalized cSNPs / 5kb")
fig.update_layout(
    yaxis_range=[0, 70],
    template="simple_white",
    #legend_title="Legend Title",
    yaxis = dict(tickfont = dict(size=25)),
    xaxis = dict(tickfont = dict(size=20)),
    #legend = dict(font = dict(size = 20, color = "black")),
    font=dict(size=30))
fig.show()

## Group patients by cancer type and tissue and calculate median G4-averages across patients with similiar diagnoses. Plot as sunburst plot. 

In [ ]:
key = "G4_n"
df_results_normalized = pd.DataFrame({"primary_diagnosis": [],
                           "tissue": [],
                           "Patient_Count": [],
                           "G4_mean": [],
                           "G4_median": [],
                           "G4_var": [],
                           "G4_std": []})

diagnoses = df_m3.primary_diagnosis.unique()

length = len(diagnoses)
i = 0
for diagnosis in diagnoses:
    tissues = df_m3[df_m3["primary_diagnosis"] == diagnosis]["tissue_or_organ_of_origin"].unique()
    for tissue in tissues:
        sub = df_m3[(df_m3["primary_diagnosis"] == diagnosis) & (df_m3["tissue_or_organ_of_origin"] == tissue)]
        sub["G4_n"] = sub["G4_Count"] * (sub["G4_Count"] / sub["G4_Count"].sum()) 
        df_now = pd.DataFrame({"primary_diagnosis": [diagnosis],
                               "tissue": [tissue],
                               "Patient_Count": [len(sub["case_id"].unique())],
                               "G4_mean": [sub[key].mean()],
                               "G4_median": [sub[key].median()],
                               "G4_var": [sub[key].var()],
                               "G4_std": [sub[key].std()]})
        df_results_normalized = pd.concat([df_results_normalized, df_now])
    i += 1
    print(i/length)

df_results_grouped_normalized = pd.DataFrame({"primary_diagnosis": [],
                           "tissue": [],
                           "Patient_Count": [],
                           "G4_mean": [],
                           "G4_median": [],
                           "G4_var": [],
                           "G4_std": []})

diagnoses = df_m3.primary_diagnosis.unique()

length = len(diagnoses)
i = 0
for diagnosis in diagnoses:
    tissues = df_m3[df_m3["primary_diagnosis"] == diagnosis]["tissue_or_organ_of_origin"].unique()
    for tissue in tissues:
        sub = df_m3[(df_m3["primary_diagnosis"] == diagnosis) & (df_m3["tissue_or_organ_of_origin"] == tissue)]
        sub["G4_n"] = sub["G4_Count"] * (sub["G4_Count"] / sub["G4_Count"].sum())
        df_now = pd.DataFrame({"primary_diagnosis": [diagnosis],
                               "tissue": [tissue],
                               "Patient_Count": [len(sub["case_id"].unique())],
                               "G4_mean": [sub.groupby(["case_id"])[key].mean().mean()],
                               "G4_median": [sub.groupby(["case_id"])[key].mean().median()],
                               "G4_var": [sub.groupby(["case_id"])[key].mean().var()],
                               "G4_std": [sub.groupby(["case_id"])[key].mean().std()]})
        df_results_grouped_normalized = pd.concat([df_results_grouped_normalized, df_now])
    i += 1
    print(i/length)
    
df_results_normalized["log_mean"] = np.log10(df_results_normalized["G4_mean"])
df_results_grouped_normalized["log_std"] = np.log10(df_results_normalized["G4_std"] / df_results_normalized["G4_mean"])
df_results_grouped_normalized["norm_std"] = df_results_normalized["G4_std"] / df_results_normalized["G4_mean"]

In [ ]:
key = "log_mean"
sub = df_results_normalized[df_results_normalized["Patient_Count"] > 200].sort_values(by=key)
sub[key] = sub[key]# * 100000

fig = px.sunburst(
    sub,
    path=["tissue", "primary_diagnosis"],
    color=key,
    color_continuous_scale='RdBu',
    #range_color=[min(sub[key]), max(sub[key])],
    color_continuous_midpoint=np.average(sub[key]),# weights=sub[key]),
    width = 800,
    height = 800)

fig.update_layout(
    template="simple_white",
    #legend_title="Legend Title",
    yaxis = dict(tickfont = dict(size=25)),
    xaxis = dict(tickfont = dict(size=25)),
    #legend = dict(font = dict(size = 20, color = "black")),
    font=dict(size=20))

fig.show()

## Group patients by cancer type and calculate intra-group mean-normalized standard deviation.

In [ ]:
sub = df_results_grouped_normalized[(df_results_grouped_normalized["Patient_Count"] > 50)].groupby(["primary_diagnosis"]).mean().reset_index().sort_values(by="norm_std", ascending=False)
# Generate some example data
np.random.seed(42)
scatter_x = sub["temp"]
scatter_y = sub["norm_std"]
box_data = sub["norm_std"]
new_min = 10
new_max = 30
min_value = sub['Patient_Count'].min()
max_value = sub['Patient_Count'].max()

sub['Normalized_Patient_Count'] = (sub['Patient_Count'] - min_value) * (new_max - new_min) / (max_value - min_value) + new_min
size = sub['Normalized_Patient_Count']
# Create the scatter plot
scatter = go.Scatter(
    x=scatter_x,
    y=scatter_y,
    mode='markers',
    marker=dict(
        size=size,
        color='rgba(50, 50, 50, 0.8)'
    ),
    name='Scatter Plot'
)

# Create the box plot
box = go.Box(
    y=box_data,
    x=scatter_x,
    fillcolor = 'rgba(0, 0, 0, 0.1)',
    name='Box Plot',
    marker=dict(
        
        color='rgba(0, 0, 0, 0.8)'
    ),
    line=dict(
        color='rgba(0, 0, 0, 0.4)'
    ),
    boxpoints=False,
    notched=True# Only show outlier points
    #jitter=0.3,  # Add some jitter for better visibility of points
    #pointpos=0  # Position of the points relative to the box
)

# Create the layout for the chart
layout = go.Layout(
    #title='Overlapping Scatter and Box Plots',
    #xaxis=dict(title='X-axis Label'),
    yaxis=dict(title='Normalized std'),
    height=500,
    width=500
)

# Create the figure and add the scatter and box plots
fig = go.Figure(data=[scatter, box], layout=layout)
fig.update_layout(
    template="simple_white",
    #legend_title="Legend Title",
    yaxis = dict(tickfont = dict(size=25)),
    xaxis = dict(tickfont = dict(size=20)),
    #legend = dict(font = dict(size = 20, color = "black")),
    font=dict(size=30))
# Show the figure
fig.show()


## Fit Cox's proportional hazards model to patients that are eeither alive or died due to cancer-related circumstances. Use cSNV-average and G4-average as potential predictor variables.  

In [ ]:
df_c = pd.read_csv("/mnt/c/Users/tilma/Desktop/WORK/SNPs/SNP_DATA/GDC_Database/clinical.tsv", sep="\t")
df_closest = pd.read_csv("ALL_GDC_exons_5kb_windows_G4s.closest.temp", sep="\t", header=None).rename(columns={0: "SNP_ID", 1: "Distance"})
temp = pd.merge(df_m3[(df_m3["cause_of_death"] == "Cancer Related") | (df_m3["cause_of_death"] == "'--") & (df_m3["vital_status"] != "Not Reported")][["SNP_ID", "G4_Count", "case_id", "SNP_Count"]], df_closest, on="SNP_ID", how="left")
temp = temp.drop_duplicates(subset="SNP_ID", keep="first")
temp["Distance"] = temp["Distance"].abs()
data2 = pd.merge(temp, df_c[["case_id", "days_to_death", "cause_of_death"]], on="case_id", how="left").drop_duplicates(keep="first")
#data2 = pd.merge(df_m3[["G4_Count", "case_id", "SNP_Count"]].drop_duplicates(keep="first"), df_c[["case_id", "days_to_death", "vital_status"]], on="case_id", how="left")
#data2['days_to_death'] = data2['days_to_death'].replace("'--", default)
df_cm = df_c
#df_cm['days_to_death'] = df_cm['days_to_death'].replace("'--", default)
data3 = pd.merge(data2[["G4_Count", "case_id", "days_to_death", "Distance", "SNP_Count"]].groupby(["case_id"]).median().reset_index(), df_cm[pd.to_numeric(df_cm['days_to_death'], errors='coerce').notnull()][["case_id", "days_to_death", "cause_of_death"]], on="case_id", how="left").drop_duplicates(subset="case_id", keep="first")
data4 = pd.merge(data2[["G4_Count", "case_id", "days_to_death", "Distance"]].groupby(["case_id"]).var().reset_index().rename(columns={"G4_Count": "G4_Count_Var", "Distance": "Distance_Var"}), data3, on="case_id", how="left").dropna()

# Assuming your preprocessed DataFrame is named 'data'

data = data4#[data4["Distance"] < 100]#.drop_duplicates(subset="case_id", keep="first")#.sample(10000)
#data = data2.sample(10000)
data = data[pd.to_numeric(data['days_to_death'], errors='coerce').notnull()]
data['event'] = data['cause_of_death'].apply(lambda x: 1 if x == "Cancer Related" else 0)
# Keep a copy of the data with primary_diagnosis for step 5
#data_with_diagnosis = data.copy()


# Remove non-numeric columns
#data = data.drop(['vital_status', 'primary_diagnosis'], axis=1)
data = data[["event", "days_to_death", "G4_Count", "SNP_Count"]]
#data = data[["event", "days_to_death", "G4_Count", "Distance"]]
# Fit the Cox proportional hazards model
cph = CoxPHFitter()
cph.fit(data, duration_col='days_to_death', event_col='event')

# Print the summary
cph.print_summary()

## Plot results of Cox's proportional hazards model.

In [ ]:
x_labels = ['SNP Average', 'G4 Average']
coef_values = [1.0, 1.46]
lower_ci = [0.99, 1.28]
upper_ci = [1.01, 1.67]

bar_colors = ['blue', 'orange']  # Choose the colors for the bars

# Create the bar plot
fig, ax = plt.subplots(figsize=(8, 3))
bars = ax.barh(x_labels, coef_values, xerr=errors, capsize=5, color=bar_colors)

# Set the background color to white
ax.set_facecolor('white')
fig.set_facecolor('white')

# Leave the left and bottom grid axes
ax.xaxis.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.yaxis.grid(False)
for pos in ['right', 'top']:
    plt.gca().spines[pos].set_visible(False)
plt.rcParams.update({
    'font.size': 15,         # default font size
    'axes.labelsize': 15,    # x and y labels
    'xtick.labelsize':15,   # x-axis tick labels
    'ytick.labelsize': 15,   # y-axis tick labels
    'legend.fontsize': 25,   # legend font size
    'figure.titlesize': 40,  # figure title font size
})
plt.ylabel("Normalized cSNPs / 5kb",fontsize=15)
plt.xlabel('G4s / 5kb',fontsize=15)
plt.savefig("output_Cox_plot.png", dpi=500)
# Display the plot
plt.show()
